In [2]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os
import csv
import pickle
import pandas as pd


In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

with open('knn_model.pkl', 'rb') as f:
    knn = pickle.load(f)
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

<img src=https://i.imgur.com/qpRACer.png/>

In [3]:
cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5, max_num_hands=2) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = cv2.flip(image, 1)
        image.flags.writeable = False
        results = hands.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  
        
        print(results)
        
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS)
                
        cv2.imshow('Hand tracking', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
cap.release()
cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [22]:
label = int(input("Label for hand gesture: "))

In [23]:
os.makedirs('data', exist_ok=True)

csv_path = 'data/finger_data.csv'
csv_file = open(csv_path, mode='a', newline='')
csv_writer = csv.writer(csv_file)

In [ ]:

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils


cap = cv2.VideoCapture(0)


hands = mp_hands.Hands(min_detection_confidence=0.8,
                       min_tracking_confidence=0.5,
                       max_num_hands=1)  

while cap.isOpened():
    ret, frame = cap.read()                    
    frame = cv2.flip(frame, 1)                   
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) 

    results = hands.process(rgb)                

    if results.multi_hand_landmarks:             
        for hand_landmarks in results.multi_hand_landmarks:
            
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            
            landmarks = []
            for lm in hand_landmarks.landmark:
                landmarks.extend([lm.x, lm.y, lm.z])

            
            landmarks.append(label)

            
            csv_writer.writerow(landmarks)

    
    cv2.putText(frame, f'Label: {label}', (10, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    
    cv2.imshow('Hand Data Collector', frame)

    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()           
cv2.destroyAllWindows() 
csv_file.close()        


In [ ]:
coords = ['x', 'y', 'z']
columns = [f"{c}{i}" for i in range(21) for c in coords]

cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5, max_num_hands=2) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = cv2.flip(image, 1)
        image.flags.writeable = False
        results = hands.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  
        
        if results.multi_hand_landmarks:
            for hand in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS)
                
                landmarks = []
                for lm in hand.landmark:
                    landmarks.extend([lm.x, lm.y, lm.z])
                landmarks_df = pd.DataFrame([landmarks], columns=columns)
                landmarks_scaled = scaler.transform(landmarks_df)
                prediction = knn.predict(landmarks_scaled)
                cv2.putText(image, f'Predicted: {prediction[0]}', (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                
        cv2.imshow('Hand tracking', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
cap.release()
cv2.destroyAllWindows()
